<a href="https://colab.research.google.com/github/ehabsameh/Final_Project/blob/main/SnapChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import needed packages
!pip install pyjanitor==0.23.1
import pandas as pd
import json
from urllib.request import urlopen
import matplotlib.pyplot as plt
import plotly.express as px
from urllib import request
import json
import numpy as np
import seaborn as sns
import janitor
import datetime
import statsmodels.api as sm
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, LabelSet
import plotly.graph_objects as go
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.transform import transform

pd.options.display.max_columns = None

In [ ]:
!pip install pystan~=2.14
!pip install fbprophet

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
path='/content/drive/MyDrive/AWS_Final_Project/usage_pre_covid_nordic.csv'
snap = pd.read_csv(path)

In [15]:
snapA = snap.groupby('event_date')['APP_APPLICATION_OPEN_UU'].sum().reset_index()

In [29]:

from fbprophet import Prophet

train_data = snap.groupby('event_date')['APP_APPLICATION_OPEN_UU'].sum().reset_index().rename(columns={'event_date': 'ds', 'APP_APPLICATION_OPEN_UU': 'y'})
train_data['ds'] = pd.to_datetime(train_data['ds'])

# Select the last 120 days of data
train_data = train_data[train_data['ds'] > train_data['ds'].max() - pd.Timedelta(days=40)]

# fit a Prophet model to the training data
model = Prophet()
model.fit(train_data)

# generate the forecast for the next 9 days
future_dates = pd.date_range(start=train_data['ds'].max()+pd.Timedelta(days=1), periods=90, freq='D')
future_data = pd.DataFrame({'ds': future_dates})

forecast = model.predict(future_data)['yhat'].values

# create a new DataFrame with the forecasted values
forecast_df1 = pd.DataFrame({'event_date': future_dates, 'APP_APPLICATION_OPEN_UU': forecast})
forecast_df1['event_date'] = forecast_df1['event_date'].apply(lambda x: x.date().strftime('%Y-%m-%d'))

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.9/dist-packages/fbprophet/forecaster.py:891: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/lib/python3.9/dist-packages/fbprophet/forecaster.py:891: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/lib/python3.9/dist-packages/fbprophet/forecaster.py:891: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [30]:

snapF = pd.concat([snapA, forecast_df1], ignore_index=True)


In [31]:
import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=snapF['event_date'], y=snapF['APP_APPLICATION_OPEN_UU'], mode='lines', name='APP_APPLICATION_OPEN_UU'))
fig.update_layout(title='Forecasted Usage', xaxis_title='Date', yaxis_title='DAY')
fig.update_traces(hoverinfo='text', hovertext=[f"event_date: {date}<br>APP_APPLICATION_OPEN_UU: {APP_APPLICATION_OPEN_UU:.2f}" for date, APP_APPLICATION_OPEN_UU in zip(snapAF['event_date'], snapAF['APP_APPLICATION_OPEN_UU'])])
fig.show()

In [ ]:
!pip install rpy2
%load_ext rpy2.ipython


In [14]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA


train_data = snap.groupby('event_date')['APP_APPLICATION_OPEN_UU'].sum().reset_index().rename(columns={'event_date': 'date', 'APP_APPLICATION_OPEN_UU': 'usage'})
train_data['date'] = pd.to_datetime(train_data['date'])

# Select the last 14 days of data
train_data = train_data[train_data['date'] > train_data['date'].max() - pd.Timedelta(days=120)]

# fit an ARIMA model to the training data
model = ARIMA(train_data.set_index('date'), order=(1, 1, 0))
model_fit = model.fit()

# generate the forecast for the next 9 days
future_dates = pd.date_range(start=train_data['date'].max()+pd.Timedelta(days=1), periods=9, freq='D')
forecast = model_fit.forecast(steps=9)[0]

# create a new DataFrame with the forecasted values
forecast_df1 = pd.DataFrame({'event_date': future_dates, 'APP_APPLICATION_OPEN_UU': forecast})
forecast_df1['event_date'] = forecast_df1['event_date'].apply(lambda x: x.date().strftime('%Y-%m-%d'))





/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.

/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.

/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.

